<a href="https://colab.research.google.com/github/AdithyaJesuman/Image_classification/blob/main/telecomchurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Customer churn prediction

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [5]:
#link=https://www.kaggle.com/datasets/blastchar/telco-customer-churn/discussion/263401
df = pd.read_csv("customer_churn.csv")
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
1846,7369-TRPFD,Male,0,No,No,19,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,18.80,279.2,No
3245,0461-CVKMU,Female,0,Yes,Yes,23,Yes,No,Fiber optic,No,...,Yes,Yes,No,No,Month-to-month,No,Electronic check,83.80,1900.25,Yes
3392,7636-PEPNS,Female,0,Yes,Yes,58,Yes,No,DSL,No,...,No,No,No,No,One year,Yes,Mailed check,44.10,2413.05,No
2827,1845-ZLLIG,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,50.15,50.15,No
444,1910-FMXJM,Female,0,Yes,No,36,Yes,No,Fiber optic,No,...,No,No,No,Yes,Month-to-month,No,Electronic check,80.40,2937.65,No


In [6]:
df.drop('customerID',inplace=True,axis='columns')

In [7]:
df.replace('No internet service','No',inplace=True)
df.replace('No phone service','No',inplace=True)

In [8]:
def uq():
  for col in df.columns:
      print(f"{col}: {df[col].unique()}\n")
uq()

gender: ['Female' 'Male']

SeniorCitizen: [0 1]

Partner: ['Yes' 'No']

Dependents: ['No' 'Yes']

tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]

PhoneService: ['No' 'Yes']

MultipleLines: ['No' 'Yes']

InternetService: ['DSL' 'Fiber optic' 'No']

OnlineSecurity: ['No' 'Yes']

OnlineBackup: ['Yes' 'No']

DeviceProtection: ['No' 'Yes']

TechSupport: ['No' 'Yes']

StreamingTV: ['No' 'Yes']

StreamingMovies: ['No' 'Yes']

Contract: ['Month-to-month' 'One year' 'Two year']

PaperlessBilling: ['Yes' 'No']

PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']

MonthlyCharges: [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]

TotalCharges: ['29.85' '1889.5' '108.15' ... '346.45' '306.6' '6844.5']

Churn: ['No' 'Yes']



In [9]:
df.drop(df[df['TotalCharges']==' '].index,inplace=True)
df.TotalCharges=pd.to_numeric(df.TotalCharges)
rp=['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
    'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']
for col in rp:
  df[col].replace({'Yes':1,'No':0},inplace=True)
df['gender'].replace({'Male':1,'Female':0},inplace=True)
df=pd.get_dummies(data=df, columns=['InternetService','Contract','PaymentMethod'])
df.columns
uq()

gender: [0 1]

SeniorCitizen: [0 1]

Partner: [1 0]

Dependents: [0 1]

tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]

PhoneService: [0 1]

MultipleLines: [0 1]

OnlineSecurity: [0 1]

OnlineBackup: [1 0]

DeviceProtection: [0 1]

TechSupport: [0 1]

StreamingTV: [0 1]

StreamingMovies: [0 1]

PaperlessBilling: [1 0]

MonthlyCharges: [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]

TotalCharges: [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]

Churn: [0 1]

InternetService_DSL: [ True False]

InternetService_Fiber optic: [False  True]

InternetService_No: [False  True]

Contract_Month-to-month: [ True False]

Contract_One year: [False  True]

Contract_Two year: [False  True]

PaymentMethod_Bank transfer (automatic): [False  True]

PaymentMethod_Credit card (automatic): [False  True]

PaymentMethod_Electroni

/tmp/ipython-input-9-241862512.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace({'Yes':1,'No':0},inplace=True)
/tmp/ipython-input-9-241862512.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col].replace({'Yes':1,'No':0},inplace=True)
/tmp/ipython-input-9-241862512.py:7: Fut

In [10]:
df.shape[1]

27

In [11]:
scale=MinMaxScaler()
col1=['tenure','MonthlyCharges','TotalCharges']
df[col1]=scale.fit_transform(df[col1])
df[col1].sample(100)

,tenure,MonthlyCharges,TotalCharges
5821,0.957746,0.080100,0.201333
6809,0.042254,0.564179,0.033453
2775,0.732394,0.415920,0.370511
5,0.098592,0.809950,0.092511
5468,0.084507,0.611443,0.065457
...,...,...,...
3535,0.887324,0.904975,0.775715
5506,0.000000,0.019403,0.000162
710,0.000000,0.012935,0.000087
3458,0.352113,0.701990,0.260276


In [12]:
from re import X
X_train,X_test,Y_train,Y_test=train_test_split(df.drop('Churn',axis='columns'),df['Churn'],test_size=0.2,random_state=5)
model= keras.Sequential([
    keras.layers.Dense(26,input_shape=(26,),activation='relu'),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
opt = keras.optimizers.Adam(learning_rate=0.02)
model.compile(
    optimizer=opt,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)
model.fit(X_train,Y_train,epochs=100)
model.evaluate(X_test,Y_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7661 - auc: 0.7872 - loss: 0.4746 - precision: 0.6071 - recall: 0.4046
Epoch 2/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7929 - auc: 0.8278 - loss: 0.4359 - precision: 0.6602 - recall: 0.4545
Epoch 3/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7948 - auc: 0.8258 - loss: 0.4394 - precision: 0.6647 - recall: 0.4747
Epoch 4/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8104 - auc: 0.8480 - loss: 0.4001 - precision: 0.6578 - recall: 0.4688
Epoch 5/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8206 - auc: 0.8485 - loss: 0.3989 - precision: 0.6982 - recall: 0.4907
Epoch 6/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8150 - auc: 0.8557 - loss: 0.3928 - precision: 0.6757 - recall: 0.4961
Epoch 7/100
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8097 - auc: 0.8507 - loss: 0.4094 - precision: 0.7313 - recall: 0.4306
Epoch 8/100
176/176 ━━━━━━━━━━━━━━

[0.5153501033782959,
 0.7810945510864258,
 0.6838235259056091,
 0.45588234066963196,
 0.8200332522392273]

In [13]:
yp = model.predict(X_test)
prediction=[]
for i in yp :
  if i>0.5 :
    prediction.append(1)
  else :
    prediction.append(0)
count = 0
Y_test = Y_test.to_numpy()
for i in range(100):
    if Y_test[i]== prediction[i]:
        count += 1
print(count)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
81


In [14]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(Y_test[:50],prediction[:50]))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91        39
           1       0.70      0.64      0.67        11

    accuracy                           0.86        50
   macro avg       0.80      0.78      0.79        50
weighted avg       0.86      0.86      0.86        50

